In [1]:
import numpy as np
import pandas as pd

In [2]:
data_set = pd.read_csv('Dataset.csv')
data_set

,Airline,SourceAirport,DestinationAirport,SourceAirport_City,SourceAirport_Country,SourceAirport_Latitude,SourceAirport_Longitude,SourceAirport_Altitude,DestinationAirport_City,DestinationAirport_Country,DestinationAirport_Latitude,DestinationAirport_Longitude,DestinationAirport_Altitude,Distance,FlyTime,Price
0,Aerocondor,Sochi International Airport,Kazan International Airport,Sochi,Russia,43.449902,39.956600,89,Kazan,Russia,55.606201,49.278702,411,1506.825604,2.371833,723.412802
1,Aerocondor,Astrakhan Airport,Kazan International Airport,Astrakhan,Russia,46.283298,48.006302,-65,Kazan,Russia,55.606201,49.278702,411,1040.438320,1.792470,490.219160
2,Aerocondor,Chelyabinsk Balandino Airport,Kazan International Airport,Chelyabinsk,Russia,55.305801,61.503300,769,Kazan,Russia,55.606201,49.278702,411,770.508500,1.457153,355.254250
3,Aerocondor,Domodedovo International Airport,Kazan International Airport,Moscow,Russia,55.408798,37.906300,588,Kazan,Russia,55.606201,49.278702,411,715.649350,1.389005,327.824675
4,Aerocondor,Belgorod International Airport,Kazan International Airport,Belgorod,Russia,50.643799,36.590099,735,Kazan,Russia,55.606201,49.278702,411,1008.253110,1.752488,474.126555
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66153,Iberworld,Lifou Airport,Tiga Airport,Lifou,New Caledonia,-20.774799,167.240005,92,Tiga,New Caledonia,-21.096100,167.804001,128,68.609280,0.585229,4.304640
66154,Iberworld,Nouméa Magenta Airport,Touho Airport,Noumea,New Caledonia,-22.258301,166.473007,10,Touho,New Caledonia,-20.790001,165.259003,10,205.972279,0.755866,72.986139
66155,Iberworld,Nouméa Magenta Airport,Ouvéa Airport,Noumea,New Caledonia,-22.258301,166.473007,10,Ouvea,New Caledonia,-20.640600,166.572998,23,180.177492,0.723823,60.088746
66156,Iberworld,Lifou Airport,Ouvéa Airport,Lifou,New Caledonia,-20.774799,167.240005,92,Ouvea,New Caledonia,-20.640600,166.572998,23,70.962955,0.588153,5.481478


In [3]:
class Node:
    def __init__(self, label: str, city : str, latitude : str, longitude : str, altitude : str):
        self.__label = label
        self.__city = city
        self.__latitude = latitude
        self.__longitude = longitude
        self.__altitude = altitude
        self.__edges = []

    @property
    def edges(self):
        return self.__edges

    @property
    def label(self):
        return self.__label
    
    @property
    def city(self):
        return self.__city

    @property
    def latitude(self):
        return self.__latitude
    
    @property
    def longitude(self):
        return self.__longitude
    
    @property
    def altitude(self):
        return self.__altitude
    
    def add_edge(self, target, weight: int):
        edge = Edge(self, target, weight)
        self.edges.append(edge)

    def __str__(self):
        return f"{self.__label}"